<a href="https://www.kaggle.com/code/hydramst/water-pollution-lr?scriptVersionId=97692724" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_validate

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/water-pollution/Submission.csv
/kaggle/input/water-pollution/Target.csv
/kaggle/input/water-pollution/Train.csv
/kaggle/input/water-pollution/Test.csv


In [2]:
DF_train = pd.read_csv('/kaggle/input/water-pollution/Train.csv', delimiter = ',')
DF_test = pd.read_csv('/kaggle/input/water-pollution/Test.csv', delimiter = ',')
Target = pd.read_csv('/kaggle/input/water-pollution/Target.csv', delimiter = ',')
Submission = pd.read_csv('/kaggle/input/water-pollution/Submission.csv', delimiter = ',')


In [3]:
DF_total = pd.concat([DF_train, DF_test])
DF_total = DF_total.drop(columns = 'Unnamed: 0', axis =1)
df = DF_total.join(Target['polution'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2440 entries, 0 to 2317
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   code             2440 non-null   object 
 1   year             2440 non-null   int64  
 2   period           2440 non-null   object 
 3   id               2440 non-null   object 
 4   Country          2440 non-null   object 
 5   tourists         2440 non-null   float64
 6   venue            2440 non-null   float64
 7   rate             2440 non-null   float64
 8   food             2440 non-null   float64
 9   glass            2440 non-null   float64
 10  metal            2440 non-null   float64
 11  other            2440 non-null   float64
 12  paper            2440 non-null   float64
 13  plastic          2440 non-null   float64
 14  leather          2440 non-null   float64
 15  green_waste      2440 non-null   float64
 16  waste_recycling  2440 non-null   float64
 17  polution      

In [4]:
# view categorical and numerical columns 

cat_columns = []
num_columns = []

for column_name in df.columns:
    if (df[column_name].dtypes == object):
        cat_columns +=[column_name]
    else:
        num_columns +=[column_name]

print('categorical columns:\t ',cat_columns, '\n len = ',len(cat_columns))

print('numerical columns:\t ',  num_columns, '\n len = ',len(num_columns))

categorical columns:	  ['code', 'period', 'id', 'Country'] 
 len =  4
numerical columns:	  ['year', 'tourists', 'venue', 'rate', 'food', 'glass', 'metal', 'other', 'paper', 'plastic', 'leather', 'green_waste', 'waste_recycling', 'polution'] 
 len =  14


In [5]:
# find duplicated rows

df.duplicated().sum() 
df = df.drop_duplicates() 
df = df.reset_index(drop=True)


In [6]:
df = df.drop(columns = ['period', 'venue', 'rate']) 

In [7]:
# OHE of categorical features

df = pd.get_dummies(df)


In [8]:
# Split back to test and train 

train = df.iloc[0:DF_train.shape[0],:]
test = df.iloc[DF_train.shape[0]:,:]
DF_train.shape[0], train.shape[0], DF_test.shape[0], test.shape[0], 

(2318, 2318, 122, 122)

In [9]:
X, y = train.values, Target['polution'].values
X_train = train.values
X_test = test.values

#y_train = Target['polution'].values.ravel()

In [10]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler  = MinMaxScaler()
scaler.fit_transform(X_train)
X_train = scaler.transform(X_train) 
X_test  = scaler.transform(X_test) 

# Ridge LR

In [11]:

model = Ridge(alpha = 0.00005, max_iter=10000)

model.fit(X_train, y_train)

B0=model.intercept_
B=model.coef_

line='{:.2f}'.format(B0)
sign=['+','-']

features_names = df.columns
for p,(fn,b) in enumerate(zip(features_names,B)):
  if np.abs(b)>0:
    line=line+sign[int(0.5*(np.sign(b)-1))]+'{:.2f}*'.format(np.abs(b))+fn



y_predict=model.predict(X_test)

print('Ошибка на тестовых данных')
print('MSE: %.1f' % mse(y_test,y_predict))
print('RMSE: %.1f' % mse(y_test,y_predict,squared=False))
print('R2 : %.4f' %  r2_score(y_test,y_predict))

Ошибка на тестовых данных
MSE: 0.0
RMSE: 0.1
R2 : -0.2123


# Cross validation

In [12]:
scoring = {'R2': 'r2',
           '-MSE': 'neg_mean_squared_error',
           '-MAE': 'neg_mean_absolute_error',
           'Max': 'max_error'}


scores = cross_validate(model, X_train, y_train,
                      scoring=scoring, cv=ShuffleSplit(n_splits=5, random_state=42) )

print('Результаты Кросс-валидации')
DF_cv_linreg = pd.DataFrame(scores)
display(DF_cv_linreg)
print('\n')
print(DF_cv_linreg.mean()[2:])
print('\n')

Результаты Кросс-валидации


,fit_time,score_time,test_R2,test_-MSE,test_-MAE,test_Max
0,0.253645,0.008779,-0.765576,-0.016686,-0.074781,-0.808774
1,0.294075,0.008910,-0.423460,-0.019164,-0.087209,-0.616903
2,0.261860,0.002401,-1.230533,-0.018751,-0.081606,-0.798582
3,0.257899,0.002433,-0.496155,-0.016537,-0.081093,-0.833049
4,0.241047,0.002413,-0.539288,-0.016437,-0.083069,-0.587859




test_R2     -0.691002
test_-MSE   -0.017515
test_-MAE   -0.081552
test_Max    -0.729034
dtype: float64




In [13]:

Submission['polution'] = y_predict[:122]


In [14]:
! rm /kaggle/working/water-pollution/Submission.csv
! mkdir /kaggle/working/water-pollution/

rm: cannot remove '/kaggle/working/water-pollution/Submission.csv': No such file or directory


In [15]:
Submission.to_csv('/kaggle/working/water-pollution/Submission.csv',index=False)

In [16]:
! mkdir ~/.kaggle
! touch ~/.kaggle/kaggle.json
! echo '{"username":"USERNAME","key": YOUR KAGGLE API }' > ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [17]:
cat ~/.kaggle/kaggle.json

{"username":"USERNAME","key": YOUR KAGGLE API }


In [18]:
#! kaggle competitions submit -f /kaggle/working/water-pollution/Submission.csv -m "Message" sf-matml-2022-regression